In [ ]:
import cProfile
import pydub
import torchaudio
import numpy as np
import torch

In [ ]:
fp_in = "/home/julian/Music/SZA - Love Galore ft. Travis Scott.mp3"
fp_wav = "/home/julian/Music/SZA.wav"
fp_wav2 = "/home/julian/Music/SZA2.wav"
fp_wav44 = "/home/julian/Music/SZA44.wav"

In [ ]:
def run1(path, save=False, resample=True):
    audio = pydub.AudioSegment.from_mp3(path)
    if resample:
        audio = audio.set_frame_rate(16000)

    if save:
        # Save sample as file
        audio.export(fp_wav2, format="wav")
    return audio

def read_wav(path):
    audio = pydub.AudioSegment.from_wav(path)
    return torch.tensor(np.frombuffer(audio._data, dtype=np.float32))

def read_wav_resample(path):
    audio = pydub.AudioSegment.from_wav(path)
    audio = audio.set_frame_rate(16000)
    return torch.tensor(np.frombuffer(audio._data))



In [ ]:
data_pydub = read_wav(fp_wav)

In [ ]:
resampler = torchaudio.transforms.Resample(orig_freq=44000, new_freq=16000)

def run2(path, save=False, resample=True):
    audio, freq = torchaudio.load(path)
    if resample:
        audio = resampler(audio)
    
    if save:
        torchaudio.save(fp_wav2, audio, 16000)
    return audio

In [ ]:
data_pydub = read_wav(fp_wav)
data_pytorch = run2(fp_wav, save=False, resample=False)

print(data_pydub.shape)
print(data_pytorch.shape)

# assert data_pydub.shape[0] * 2 == data_pytorch.shape[1]
# print("passed")

# assert torch.allclose(data_pydub, data_pytorch[0,])
# print("passed2")
idx = 2458500

print(data_pydub[idx])
data_pytorch[1, idx]

In [ ]:
cProfile.run("run1(fp_in, save=True, resample=False)")

In [ ]:
cProfile.run("run2(fp_in, fp_wav2)")

In [ ]:
cProfile.run('run1(fp_wav, fp_wav2)')

In [ ]:
res, val = torchaudio.load(fp_wav)
res.shape, val

In [ ]:
torchaudio.info(fp)
cProfile.run('torchaudio.load(fp_wav)')

In [ ]:
def _process_audio(self, root: str, file: str) -> bool:
    """Returns True if sample was dropped due to being too long."""
    path = os.path.join(root, "clips", file)
    si, _ = torchaudio.info(path)
    duration = (si.length / si.channels) / si.rate
    if self.max_duration is not None and duration > self.max_duration:
        return True

    down_file = file[:-3] + "wav"
    down_path = os.path.join(root, "clips_down", down_file)
    if not os.path.isfile(down_path):
        # Downsample to 16KHz sample rate for consistency with other data
        # sets. Not using torchaudio and SoxEffectChain for this since it
        # is not thread safe
        audio = pydub.AudioSegment.from_mp3(path)
        audio = audio.set_frame_rate(16000)

        # Save sample as file
        audio.export(down_path, format="wav")
    self.paths.append(down_path)
    self.durations.append(duration)
    return False